# Signals, wires and operations

A `Signal` is basically a virtual pin or wire that is event sensitive. It can be abstracted by an integer or a single bit. The latter abstraction is the actual wire type, the signal is merely a container. When we create a signal, we specify a wire type and a bit length or size, normally.

Cyrite by default uses an integer type class that represents defined bit fields of a definite length. It is not entirely, but in most scenarios compatible to the MyHDL intbv class and is also named so. We inspect some of the internals by creating signals:

In [1]:
from cyhdl import *

a = Signal(intbv(2)[6:], name = "a")
b = Signal(intbv(2)[6:], name = "b")

We have given the signals explicit names, as Python does not actually have knowledge of an objects symbol per se.
The signal is initialized with the `intbv` wire of initial value `2`. The size is determined here by the slice notation.

We now create a simple added expression entity as follows:

In [2]:
expr_add = a + b
expr_add

ADD(a, b)

This has generated an internal logic representation that can be emitted to a target. If we wish to inspect and evaluate it in a sequential way, we have to actually initialize the signals by using the IRL internal `.set` method:

In [3]:
seq = a.set(4), b.set(5), expr_add
seq

(a <= 4, b <= 5, ADD(a, b))

We can now run this sequence as follows:

In [4]:
for s in seq:
    print(s, "=", ret := s.evaluate())
    
assert ret == a.evaluate() + b.evaluate()

a <= 4 = 4
b <= 5 = 5
ADD(a, b) = 9


We have now done a trivial verification of the addition by running the same operation natively in python as well as in the IRL domain.

We check the size of the result against the operand's:

In [5]:
a.size(), expr_add.size()

(6, 7)

This is expected: An addition operation always extends the operands size by one in the result.

## Operations

Standard `Signal` types support a number of operations:
   * Initialization, signed or unsigned
   * Addition, Subtraction
   * Multiplication
   * Slicing, resizing (normally implicit)
   * Shifting by **constant** values
   * Signed operations: All the above with signed values
   
The thumb rule is as follows: Operations done with Python `bv` types of the class `BuiltinIntType` behave like integers for the above arithmetic operations, however maintain a bit length. Boundary checks of the wire type may be in place.

Also, implicit truncation can occur, for example with a simple operation such as:
`a.next = a + 1`, as we saw above. This will with the standard signal types throw a warning only.

### Signed

An intbv() wire can be initialized `min` and `max` value or with an explicit slice notation, like a Verilog declaration.
If `min` is < 0, it's a signed value. To check that, we can use:

In [6]:
s = Signal(intbv(min=-8, max=7))
s.is_signed()

True

### Basic operations:

We have done an addition above, likewise, we can subtract:

In [7]:
a + b, a - b.signed()

(ADD(a, b), SUB(a, SGN(<b>)))

A multiplication:

In [8]:
ops = (ma := a * b, mb := a * b.signed())
ma.evaluate(), ma.size(), mb.evaluate(), mb.size()

(20, 12, 20, 12)

## Slicing and resizing

Resizing and extending is a very controversial topic and VHDL and Verilog handle this matter very differently, especially when it comes to signed extension.

We define a signal that can handle signed values in 12 bits. Then we define an assignment as an action expression:

In [9]:
c = Signal(intbv()[12:].signed())

action_s = c.set(b[4:].signed())

The sequence below sets `b` and performs the extension to the assigned destination size:

In [10]:
seq = b.set(0x2e), action_s
for i in seq:
    ret = i.evaluate()

ret

intbv(-2)

The MSB of the slice is '1'. Thus, when interpreted as signed value, it will evaluate as `-2`. 

In [11]:
bin(b[4:].evaluate())

'0b1110'

The sign extension for `c` produces the sign extended bit vector as follows (we need to create a mask to display the negative value in its raw bit representation):

In [12]:
bin(ret & ((1 << c.size()) -1))

'0b111111111110'

To assign this to an unsigned signal, we need to cast using `.unsigned()`. If we don't do that, the myhdl_intbv compatible range guard will complain.

In [13]:
cu = Signal(intbv()[12:])
v = c.unsigned()
cu.set(v).evaluate()

bin(cu.evaluate())

'0b111111111110'

## Auxiliary inspection tips

A developer with a HDL background might want to see what the expression correponds to in VHDL or Verilog.

This can be done by a dummy target context instance that is specified for the `.emit` method of a convertible expression.

In [14]:
from myirl.targets.dummy import DummyTargetModule

d = DummyTargetModule(targets.VHDL)

action_s.emit(d)

s_4248 <= signed(resize(signed(b(4-1 downto 0)), 12));


## Signal type extensions

Shift operations with non-constant values are not covered by the standard `Signal` class. The reason is, that this is no longer a trivial operation, it may actually infer complex logic. Therefore there are library extensions inheriting from the `Signal` class that support custom operations.

### SSignal: Barrel shifter support

Since the shift operation with variable width is more complex and will silently instance hardware inside the hierarchy, it can no longer be simply expressed in an interactive environment. We have to create a test unit.
We can use an extended type in the signal interface, but then we can not pass a standard signal type. We therefore create an internal copy of the input as a `SSignal` type.
For asynchronous direkt output, we use a continous assignment, see `connections`.

In [15]:
from myirl.library.shift import SSignal


@block
def shifter_unit(a : Signal, q : Signal.Output, sa : Signal):
    
    sz = q.size()
    ia = SSignal(intbv(1)[sz:])

    connections = [
        ia    @assign@   a,
        q     @assign@   (ia << sa)
    ]
    
    return instances()

Then we create an instance using 16 bit shift registers:

In [16]:
sa = Signal(intbv()[4:], name = 'sa')
u, v = [ Signal(intbv(1)[16:], name = n) for n in "ab" ]
uut = shifter_unit(u, v, sa)

 DEBUG Inline builtin instance [block_inline 'bshifter_inline/bshifter_inline'] 
 Declare obj 'bshifter_inline' in context '(EmulationModule 'shifter_unit')'(<class 'myirl.emulation.myhdl2irl.EmulationModule'>) 
 Module shifter_unit: Existing instance bs_stage, rename to bs_stage_1 
 Module shifter_unit: Existing instance bs_stage, rename to bs_stage_2 
 Module shifter_unit: Existing instance bs_stage, rename to bs_stage_3 


To see the Verilog result, we elaborate as follows:

In [17]:
f = uut.elab(targets.Verilog)

 Writing 'shifter_unit' to file /tmp/myirl_shifter_unit_vmx_p1w1/shifter_unit.v 
DEBUG Fallback wire for sa


The generated HDL file:

In [18]:
!cat {f[0]}

// File generated from source:
//     /tmp/ipykernel_1367/29456133.py
// (c) 2016-2022 section5.ch
// Modifications may be lost, edit the source file instead.

`timescale 1 ns / 1 ps
`include "aux.v"
// Architecture cyriteHDL

module shifter_unit
    (
        input wire [15:0] a,
        output wire [15:0] q,
        input wire [3:0] sa
    );
    // Local type declarations
    // Signal declarations
    wire [15:0] s_9558;
    wire [15:0] ia;
    assign  ia = a;
    assign  q = s_9558;
    
    // Instance bshifter_inline
    bshifter_inline
        bshifter_inline_1
        (
            .d(ia),
            .sh(sa),
            .result(s_9558)
        );
endmodule // shifter_unit


This exposes a problem, which leads us to the following...


**Issues**:
*  Ony inside `@always(clk.posedge)` process for clock synchronous output. Otherwise use a direct connection.

If we did use a process that is sensitive to the input, we would actually create a faulty behaviour model, as auxiliary signals are inserted. This would not be a problem for synthesis, but for HDL simulation. Therefore the inline shifter unit creation will throw an exception when used in an asychronous process.

## Standard logic and Tristate

In hardware, we normally have '0' and '1' as a logic state. However with bidirectional pins, we require a third, 'Z' state, depending on an implicit driver.

When dealing with uninitialized values or special simulation constructs, the standard logic from VHDL or Verilog may come in handy. These scenarios require extra extended Signal types as well.

See [Standard logic](stdlogic.ipynb) for more details.